<a href="https://colab.research.google.com/github/gchn24/DS-Notes/blob/main/Copy_of_Pandas4_RevisionNotes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Content

- **Merge**
    - Concat v/s Merge
    - `left_on` and `right_on`
    - Joins


- **Intoduction to IMDB dataset**
    - Reading two datasets

- **Merging the dataframes**
    - `unique()` and `nunique()`
    - `isin()`
    - Using Left Join for `merge()`

- **Feature Exploration**
    - Create new features
   

- **Fetching data using pandas**
    - Quering from dataframe - Masking, Filtering, `&` and `|`
    

- **Apply**


In [ ]:
import pandas as pd 
import numpy as np

## Merging Dataframes

So far we have only concatenated and not merged data

#### But what is the difference between concat and merge ?

  - `concat`
    - simply stacks multiple DataFrame together along an axis

  - `merge`
    - combines dataframes in a **smart** way based on values in shared columns

  <img src="https://miro.medium.com/max/842/1*0wu6DunCzPC4o9FIyRTW4w.png" height = 200/>

  <img src="https://miro.medium.com/max/842/1*-uSHoxrzM57syqnKnms2iA.png" height = 200/>







#### How can we know the **name of the person who sent a particular message?**

We need information from **both the dataframes** 

So can we use pd.concat() for combining the dataframes ?

**No**




In [ ]:
users = pd.DataFrame({"userid":[1, 2, 3], "name":["sharadh", "shahid", "khusalli"]})
users

,userid,name
0,1,sharadh
1,2,shahid
2,3,khusalli


In [ ]:
msgs = pd.DataFrame({"userid":[1, 1, 2, 4], "msg":['hmm', "acha", "theek hai", "nice"]})
msgs

,userid,msg
0,1,hmm
1,1,acha
2,2,theek hai
3,4,nice


In [ ]:
pd.concat([users, msgs], axis=1)

,userid,name,userid,msg
0,1.0,sharadh,1,hmm
1,2.0,shahid,1,acha
2,3.0,khusalli,2,theek hai
3,NaN,NaN,4,nice


What are the problems with concat here?
- `concat` simply **combined/stacked the dataframe horizontally**
- If you notice, `userid 3` for **user** dataframe is stacked against `userid 2` for msg dataframe
- This way of stacking **doesn't help us gain any insights**

=> pd.concat() does not work according to the values in the columns

We need to **merge** the data

#### How can we join the dataframes ?

In [ ]:
users.merge(msgs, on="userid")

,userid,name,msg
0,1,sharadh,hmm
1,1,sharadh,acha
2,2,shahid,theek hai


Notice that `users` has a userid = 3 but `msgs` does not
  - When we **merge** these dataframes the **userid = 3 is not included**
  - Similarly, **userid = 4 is not present** in `users`, and thus **not included**
  - Only the userid **common in both dataframes** is shown

What type of join is this?

**Inner Join**

#### Remember joins from SQL?

<img src="https://www.datasciencemadesimple.com/wp-content/uploads/2017/09/join-or-merge-in-python-pandas-1.png?ezimgfmt=ng:webp/ngcb1" />

The `on` parameter specifies the `key`, similar to `primary key` in SQL

#### Now what join we want to use to get info of all the users and all the messages?
  


In [ ]:
users.merge(msgs, on = "userid", how="outer")

,userid,name,msg
0,1,sharadh,hmm
1,1,sharadh,acha
2,2,shahid,theek hai
3,3,khusalli,NaN
4,4,NaN,nice


Note: 

All missing values are replaced with `NaN`

#### And what if we want the info of all the users in the dataframe?


In [ ]:
users.merge(msgs, on = "userid",how="left") 

,userid,name,msg
0,1,sharadh,hmm
1,1,sharadh,acha
2,2,shahid,theek hai
3,3,khusalli,NaN


#### Similarly, what if we want all the messages and info only for the users who sent a message?

In [ ]:
users.merge(msgs, on = "userid", how="right") 

,userid,name,msg
0,1,sharadh,hmm
1,1,sharadh,acha
2,2,shahid,theek hai
3,4,NaN,nice


Note,

**NaN** in **name** can be thought of as an anonymous message

But sometimes the column names might be different even if they contain the same data





Let's rename our users column `userid` to `id`

In [ ]:
users.rename(columns = {"userid": "id"}, inplace = True)
users

,id,name
0,1,sharadh
1,2,shahid
2,3,khusalli


#### Now, how can we merge the 2 dataframes when the `key` has a different name ?

In [ ]:
users.merge(msgs, left_on="id", right_on="userid") 

,id,name,userid,msg
0,1,sharadh,1,hmm
1,1,sharadh,1,acha
2,2,shahid,2,theek hai


Here, 

- `left_on`: Specifies the **key of the 1st dataframe** (users here)

- `right_on`: Specifies the **key of the 2nd dataframe** (msgs here)

## IMDB Movie Business Use-case





#### Imagine you are working as a Data Scientist for an Analytics firm

Your task is to analyse some **movie trends** for a client 

**IMDB** has online database of information related to movies

The database contains info of several years about:
  - Movies
  - Rating
  - Director
  - Popularity
  - Revenue & Budget


#### Lets download and read the IMDB dataset

  - File1: https://drive.google.com/file/d/1s2TkjSpzNc4SyxqRrQleZyDIHlc7bxnd/view?usp=sharing
  - File2: https://drive.google.com/file/d/1Ws-_s1fHZ9nHfGLVUQurbHDvStePlEJm/view?usp=sharing

In [ ]:
!gdown 1s2TkjSpzNc4SyxqRrQleZyDIHlc7bxnd -O movies.csv

Downloading...
From: https://drive.google.com/uc?id=1s2TkjSpzNc4SyxqRrQleZyDIHlc7bxnd
To: /content/movies.csv
100% 112k/112k [00:00<00:00, 79.3MB/s]


In [ ]:
!gdown 1Ws-_s1fHZ9nHfGLVUQurbHDvStePlEJm -O directors.csv

Downloading...
From: https://drive.google.com/uc?id=1Ws-_s1fHZ9nHfGLVUQurbHDvStePlEJm
To: /content/directors.csv
100% 65.4k/65.4k [00:00<00:00, 52.1MB/s]


Here we have two csv files 
- `movies.csv`
- `directors.csv`

In [ ]:
movies = pd.read_csv('movies.csv')
#Top 5 rows
movies.head()

,Unnamed: 0,id,budget,popularity,revenue,title,vote_average,vote_count,director_id,year,month,day
0,0,43597,237000000,150,2787965087,Avatar,7.2,11800,4762,2009,Dec,Thursday
1,1,43598,300000000,139,961000000,Pirates of the Caribbean: At World's End,6.9,4500,4763,2007,May,Saturday
2,2,43599,245000000,107,880674609,Spectre,6.3,4466,4764,2015,Oct,Monday
3,3,43600,250000000,112,1084939099,The Dark Knight Rises,7.6,9106,4765,2012,Jul,Monday
4,5,43602,258000000,115,890871626,Spider-Man 3,5.9,3576,4767,2007,May,Tuesday



#### So what kind of questions can we ask from this dataset?
   - **Top 10 most popular movies**, using `popularity`
   - Or find some **highest rated movies**, using `vote_average`
   - We can find number of **movies released per year** too
   - Or maybe we can find **highest budget movies in a year** using both `budget` and `year`

#### But can we ask more interesting/deeper questions? 
   - Do you think we can find the **most productive directors**?
   - Which **directors produce high budget films**?
   - **Highest and lowest rated movies for every month** in a particular year?






Notice, there's a column **Unnamed: 0** which represents nothing but the index of a row. 

#### How to get rid of this `Unnamed: 0` col?

In [ ]:
movies = pd.read_csv('movies.csv', index_col=0)
movies.head()

,id,budget,popularity,revenue,title,vote_average,vote_count,director_id,year,month,day
0,43597,237000000,150,2787965087,Avatar,7.2,11800,4762,2009,Dec,Thursday
1,43598,300000000,139,961000000,Pirates of the Caribbean: At World's End,6.9,4500,4763,2007,May,Saturday
2,43599,245000000,107,880674609,Spectre,6.3,4466,4764,2015,Oct,Monday
3,43600,250000000,112,1084939099,The Dark Knight Rises,7.6,9106,4765,2012,Jul,Monday
5,43602,258000000,115,890871626,Spider-Man 3,5.9,3576,4767,2007,May,Tuesday


`index_col=0` explicitly states to treat the first column as the index

The default value is `index_col=None`


In [ ]:
movies.shape

(1465, 11)

The movies df contains 1465 rows,11 columns

#### Lets read the `directors` dataset:

In [ ]:
directors = pd.read_csv('directors.csv',index_col=0)
directors.head()

,director_name,id,gender
0,James Cameron,4762,Male
1,Gore Verbinski,4763,Male
2,Sam Mendes,4764,Male
3,Christopher Nolan,4765,Male
4,Andrew Stanton,4766,Male


In [ ]:
directors.shape

(2349, 3)

Directors df contains:

2349 rows,3 columns


#### **Summary**

1. Movie dataset contains info about **movies, release, popularity, ratings and the director ID**
2. Director dataset contains **detailed info about the director**

### Merging the director and movie data



#### Now, how can we know the details about the Director of a particular movie?

We will have to merge these datasets


#### So on which column we should merge the dfs ?

We will use the **ID columns** (representing unique director) in both the datasets

If you observe,

=> `director_id` of movies are taken from `id` of directors dataframe

Thus we can merge our dataframes based on these two columns as **keys**

Before that, lets first check number of unique director values in our `movies` data

#### How do we get the number of unique directors in `movies`?


In [ ]:
movies['director_id'].nunique()

199

Recall, 

we had learnt about nunique earlier

Similarly for unique diretors in `directors` df

In [ ]:
directors['id'].nunique()

2349

Summary:
- Movies Dataset: 1465 rows, but only 199 unique directors
- Directors Dataset: 2349 unique directors (= no of rows)


#### What can we infer from this?

=> Directors in `movies` is a subset of directors in `directors`


#### Now, how can we check if all `director_id` values are present in `id`?





  

In [ ]:
movies['director_id'].isin(directors['id'])

0       True
1       True
2       True
3       True
5       True
        ... 
4736    True
4743    True
4748    True
4749    True
4768    True
Name: director_id, Length: 1465, dtype: bool

The `isin()` method checks if the Dataframe column contains the specified value(s).

#### How is `isin` different from Python `in`?

- `in` works for **one element** at a time
- `isin` does this for **all the values** in the column


If you notice,
- This is like a boolean "mask"
- It returns a df similar to the original df
- For rows with values of `director_id` present in `id` it returns True, else False

#### How can we check if there is any False here?
 



In [ ]:
np.all(movies['director_id'].isin(directors['id']))

True

Lets finally merge our dataframes

Do we need to keep **all the rows for movies**?

**YES**

Do we need to keep **all the rows of directors**?

**NO**
- only the ones for which we have a corresponding row in movies

#### So which `join` type do you think we should apply here ?

We can use  LEFT JOIN






In [ ]:
data = movies.merge(directors, how='left', left_on='director_id',right_on='id')  
data

,id_x,budget,popularity,revenue,title,vote_average,vote_count,director_id,year,month,day,director_name,id_y,gender
0,43597,237000000,150,2787965087,Avatar,7.2,11800,4762,2009,Dec,Thursday,James Cameron,4762,Male
1,43598,300000000,139,961000000,Pirates of the Caribbean: At World's End,6.9,4500,4763,2007,May,Saturday,Gore Verbinski,4763,Male
2,43599,245000000,107,880674609,Spectre,6.3,4466,4764,2015,Oct,Monday,Sam Mendes,4764,Male
3,43600,250000000,112,1084939099,The Dark Knight Rises,7.6,9106,4765,2012,Jul,Monday,Christopher Nolan,4765,Male
4,43602,258000000,115,890871626,Spider-Man 3,5.9,3576,4767,2007,May,Tuesday,Sam Raimi,4767,Male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,48363,0,3,321952,The Last Waltz,7.9,64,4809,1978,May,Monday,Martin Scorsese,4809,Male
1461,48370,27000,19,3151130,Clerks,7.4,755,5369,1994,Sep,Tuesday,Kevin Smith,5369,Male
1462,48375,0,7,0,Rampage,6.0,131,5148,2009,Aug,Friday,Uwe Boll,5148,Male
1463,48376,0,3,0,Slacker,6.4,77,5535,1990,Jul,Friday,Richard Linklater,5535,Male


Notice, two stranger id columns `id_x` and `id_y`.

#### What do you think these newly created cols are?

Since the columns with name `id`is present in both the df

- `id_x` represents **id values from movie df**
- `id_y` represents **id values from directors df**

#### Do you think any column is redundant here and can be dropped?

  - `id_y` is redundant as it is same as `director_id`
  - But we dont require  `director_id` further

So we can simply drop these features

In [ ]:
data.drop(['director_id','id_y'],axis=1,inplace=True)
data.head()

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender
0,43597,237000000,150,2787965087,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,Male
1,43598,300000000,139,961000000,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,Male
2,43599,245000000,107,880674609,Spectre,6.3,4466,2015,Oct,Monday,Sam Mendes,Male
3,43600,250000000,112,1084939099,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,Male
4,43602,258000000,115,890871626,Spider-Man 3,5.9,3576,2007,May,Tuesday,Sam Raimi,Male


## Feature Exploration


Lets explore all the features in the merged dataset



In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1465 entries, 0 to 1464
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id_x           1465 non-null   int64  
 1   budget         1465 non-null   int64  
 2   popularity     1465 non-null   int64  
 3   revenue        1465 non-null   int64  
 4   title          1465 non-null   object 
 5   vote_average   1465 non-null   float64
 6   vote_count     1465 non-null   int64  
 7   year           1465 non-null   int64  
 8   month          1465 non-null   object 
 9   day            1465 non-null   object 
 10  director_name  1465 non-null   object 
 11  gender         1341 non-null   object 
dtypes: float64(1), int64(6), object(5)
memory usage: 148.8+ KB


Looks like only `gender` column has missing values (will come later)

#### How can we describe these features to know more about their range of values?

In [ ]:
data.describe()

,id_x,budget,popularity,revenue,vote_average,vote_count,year
count,1465.000000,1.465000e+03,1465.000000,1.465000e+03,1465.000000,1465.000000,1465.000000
mean,45225.191126,4.802295e+07,30.855973,1.432539e+08,6.368191,1146.396587,2002.615017
std,1189.096396,4.935541e+07,34.845214,2.064918e+08,0.818033,1578.077438,8.680141
min,43597.000000,0.000000e+00,0.000000,0.000000e+00,3.000000,1.000000,1976.000000
25%,44236.000000,1.400000e+07,11.000000,1.738013e+07,5.900000,216.000000,1998.000000
50%,45022.000000,3.300000e+07,23.000000,7.578164e+07,6.400000,571.000000,2004.000000
75%,45990.000000,6.600000e+07,41.000000,1.792469e+08,6.900000,1387.000000,2009.000000
max,48395.000000,3.800000e+08,724.000000,2.787965e+09,8.300000,13752.000000,2016.000000


This gives us all **statistical properties** of the columns

If you notice, some columns such as "title", "month" are missing

How are these **missing columns different?**

They are of **object dtype**

#### Then how can we include object type in `df.describe()`?

In [ ]:
data.describe(include=object)

,title,month,day,director_name,gender
count,1465,1465,1465,1465,1341
unique,1465,12,7,199,2
top,Avatar,Dec,Friday,Steven Spielberg,Male
freq,1,193,654,26,1309


If you notice,

- The range of values in the `revenue` and `budget` seem to be very high
- Generally budget and revenue for Hollywood movies is in million dollars


<!-- So it will be better to change the values into `million dollars USD`  -->

#### How can we change the values of `revenue` and `budget` into million dollars USD?


In [ ]:
data['revenue'] = (data['revenue']/1000000).round(2)
data


,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender
0,43597,237000000,150,2787.97,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,Male
1,43598,300000000,139,961.00,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,Male
2,43599,245000000,107,880.67,Spectre,6.3,4466,2015,Oct,Monday,Sam Mendes,Male
3,43600,250000000,112,1084.94,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,Male
4,43602,258000000,115,890.87,Spider-Man 3,5.9,3576,2007,May,Tuesday,Sam Raimi,Male
...,...,...,...,...,...,...,...,...,...,...,...,...
1460,48363,0,3,0.32,The Last Waltz,7.9,64,1978,May,Monday,Martin Scorsese,Male
1461,48370,27000,19,3.15,Clerks,7.4,755,1994,Sep,Tuesday,Kevin Smith,Male
1462,48375,0,7,0.00,Rampage,6.0,131,2009,Aug,Friday,Uwe Boll,Male
1463,48376,0,3,0.00,Slacker,6.4,77,1990,Jul,Friday,Richard Linklater,Male


Similarly, we can do it for 'budget' as well

In [ ]:
data['budget']=(data['budget']/1000000).round(2)
data.head()

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender
0,43597,237.0,150,2787.97,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,Male
1,43598,300.0,139,961.00,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,Male
2,43599,245.0,107,880.67,Spectre,6.3,4466,2015,Oct,Monday,Sam Mendes,Male
3,43600,250.0,112,1084.94,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,Male
4,43602,258.0,115,890.87,Spider-Man 3,5.9,3576,2007,May,Tuesday,Sam Raimi,Male


## Fetching queries from dataframe



Lets say we are interested in fetching all **highly rated movies**
- say movies with **ratings > 7**

#### How can we get movies with ratings > 7?

We can use the concept of `masking`

<!-- -  Have you ever come across or seen SQL?
- For now, just know that **SQL is used to fetch data from databases**
- We can use basic **Pandas operations** in a similar way to **fetch desired data from loaded data** -->


Lets first create a mask to filter such movies 
- In SQL: ```SELECT * FROM movies WHERE vote_average > 7```
- In pandas:

In [ ]:
data['vote_average'] > 7

0        True
1       False
2       False
3        True
4       False
        ...  
1460     True
1461     True
1462    False
1463    False
1464    False
Name: vote_average, Length: 1465, dtype: bool

But we still don't know the row values ... Only that which row satisfied the condtion

#### How do we get the row values from this mask?

In [ ]:
data.loc[data['vote_average'] > 7]

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender
0,43597,237.00,150,2787.97,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,Male
3,43600,250.00,112,1084.94,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,Male
14,43616,250.00,120,956.02,The Hobbit: The Battle of the Five Armies,7.1,4760,2014,Dec,Wednesday,Peter Jackson,Male
16,43619,250.00,94,958.40,The Hobbit: The Desolation of Smaug,7.6,4524,2013,Dec,Wednesday,Peter Jackson,Male
19,43622,200.00,100,1845.03,Titanic,7.5,7562,1997,Nov,Tuesday,James Cameron,Male
...,...,...,...,...,...,...,...,...,...,...,...,...
1456,48321,0.01,20,7.00,Eraserhead,7.5,485,1977,Mar,Saturday,David Lynch,Male
1457,48323,0.00,5,0.00,The Mighty,7.1,51,1998,Oct,Friday,Peter Chelsom,Male
1458,48335,0.06,27,3.22,Pi,7.1,586,1998,Jul,Friday,Darren Aronofsky,Male
1460,48363,0.00,3,0.32,The Last Waltz,7.9,64,1978,May,Monday,Martin Scorsese,Male



You can also perform the filtering without even using `loc`




In [ ]:
data[data['vote_average'] > 7]

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender
0,43597,237.00,150,2787.97,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,Male
3,43600,250.00,112,1084.94,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,Male
14,43616,250.00,120,956.02,The Hobbit: The Battle of the Five Armies,7.1,4760,2014,Dec,Wednesday,Peter Jackson,Male
16,43619,250.00,94,958.40,The Hobbit: The Desolation of Smaug,7.6,4524,2013,Dec,Wednesday,Peter Jackson,Male
19,43622,200.00,100,1845.03,Titanic,7.5,7562,1997,Nov,Tuesday,James Cameron,Male
...,...,...,...,...,...,...,...,...,...,...,...,...
1456,48321,0.01,20,7.00,Eraserhead,7.5,485,1977,Mar,Saturday,David Lynch,Male
1457,48323,0.00,5,0.00,The Mighty,7.1,51,1998,Oct,Friday,Peter Chelsom,Male
1458,48335,0.06,27,3.22,Pi,7.1,586,1998,Jul,Friday,Darren Aronofsky,Male
1460,48363,0.00,3,0.32,The Last Waltz,7.9,64,1978,May,Monday,Martin Scorsese,Male


But this is not recommended. Why ?
  - It can create a confusion between implicit/explicit indexing used as discussed before
  - `loc` is also much faster


#### Now, how can we return a subset of columns, say, only `title` and `director_name`?

In [ ]:
data.loc[data['vote_average'] > 7, ['title','director_name']]

,title,director_name
0,Avatar,James Cameron
3,The Dark Knight Rises,Christopher Nolan
14,The Hobbit: The Battle of the Five Armies,Peter Jackson
16,The Hobbit: The Desolation of Smaug,Peter Jackson
19,Titanic,James Cameron
...,...,...
1456,Eraserhead,David Lynch
1457,The Mighty,Peter Chelsom
1458,Pi,Darren Aronofsky
1460,The Last Waltz,Martin Scorsese


So far we saw only single condition for filtering 

#### What if we want to filter highly rated movies released after 2014?

Notice that two conditions are involved here
1. Movies need to be highly rated i.e.. > 7
2. They should be 2015 and onwards


We can **use AND operator b/w multiple conditions**

In [ ]:
data.loc[(data['vote_average'] > 7) & (data['year'] >= 2015)].head()

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender
30,43641,190.0,102,1506.25,Furious 7,7.3,4176,2015,Apr,Wednesday,James Wan,Male
78,43724,150.0,434,378.86,Mad Max: Fury Road,7.2,9427,2015,May,Wednesday,George Miller,Male
106,43773,135.0,100,532.95,The Revenant,7.3,6396,2015,Dec,Friday,Alejandro González Iñárritu,Male
162,43867,108.0,167,630.16,The Martian,7.6,7268,2015,Sep,Wednesday,Ridley Scott,Male
312,44128,75.0,48,108.15,The Man from U.N.C.L.E.,7.1,2265,2015,Aug,Thursday,Guy Ritchie,Male


Recall how we apply **mutliple conditions in numpy ?**

Use **elementwise operator `&` or `|`**

Note:
-  **we cannot use `and` or `or`** with dataframe 
<!-- as a  dataframe has **multiple values** -->
- **for multiple conditions**, we need to put each **separate condition within parenthesis `()`**

#### Similarly how can we find movies released on either Friday or Sunday?

In [ ]:
data.loc[(data['day'] == 'Friday') | (data['day'] == 'Saturday')].head()

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender
1,43598,300.0,139,961.00,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,Male
12,43614,380.0,135,1045.71,Pirates of the Caribbean: On Stranger Tides,6.4,4948,2011,May,Saturday,Rob Marshall,Male
22,43627,200.0,35,783.77,Spider-Man 2,6.7,4321,2004,Jun,Friday,Sam Raimi,Male
25,43632,150.0,21,836.30,Transformers: Revenge of the Fallen,6.0,3138,2009,Jun,Friday,Michael Bay,Male
40,43656,200.0,45,769.65,2012,5.6,4903,2009,Oct,Saturday,Roland Emmerich,Male


Thus we can do complex queries using both ```&``` and ```|``` operators 

Now let's try to answer few more Questions from this data

#### How will you find Top 5 most popular movies?

We can simply sort our data based on values of column 'popularity'

In [ ]:
data.sort_values(['popularity'],ascending=False).head(5)

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender
58,43692,165.0,724,675.12,Interstellar,8.1,10867,2014,Nov,Wednesday,Christopher Nolan,Male
78,43724,150.0,434,378.86,Mad Max: Fury Road,7.2,9427,2015,May,Wednesday,George Miller,Male
119,43796,140.0,271,655.01,Pirates of the Caribbean: The Curse of the Bla...,7.5,6985,2003,Jul,Wednesday,Gore Verbinski,Male
120,43797,125.0,206,752.10,The Hunger Games: Mockingjay - Part 1,6.6,5584,2014,Nov,Tuesday,Francis Lawrence,Male
45,43662,185.0,187,1004.56,The Dark Knight,8.2,12002,2008,Jul,Wednesday,Christopher Nolan,Male


On applying this to a string column, it sorts the dataframe ***lexicographically**

In [ ]:
data.sort_values(['title'],ascending=False).head(5)

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender
436,44364,60.0,36,71.07,xXx: State of the Union,4.7,549,2005,Apr,Wednesday,Lee Tamahori,Male
330,44165,70.0,46,277.45,xXx,5.8,1424,2002,Aug,Friday,Rob Cohen,Male
994,45681,15.0,21,2.86,eXistenZ,6.7,475,1999,Apr,Wednesday,David Cronenberg,Male
547,44594,50.0,37,55.97,Zoolander 2,4.7,797,2016,Feb,Saturday,Ben Stiller,Male
850,45313,28.0,38,60.78,Zoolander,6.1,1337,2001,Sep,Friday,Ben Stiller,Male


#### Now, how will get list of movies directed by a particular director, say, 'Christopher Nolan'?

In [ ]:
data.loc[data['director_name'] == 'Christopher Nolan',['title']]

,title
3,The Dark Knight Rises
45,The Dark Knight
58,Interstellar
59,Inception
74,Batman Begins
565,Insomnia
641,The Prestige
1341,Memento


Note: 
- The string indicating "Christopher Nolan" could have been something else as well.  
- The better way is to use string methods, we will discuss this later

## Apply

Now suppose we want to convert our `Gender` column data to numerical format

Basically, 
- 0 for Male
- 1 for Female

#### How can we encode the column?

Let's first write a function to do it for a single value




In [ ]:
def encode(data):
  if data == "Male":
    return 0
  else:
    return 1 

#### Now how can we apply this function to the whole column?


In [ ]:
data['gender'] = data['gender'].apply(encode)
data

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender
0,43597,237.00,150,2787.97,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,0
1,43598,300.00,139,961.00,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,0
2,43599,245.00,107,880.67,Spectre,6.3,4466,2015,Oct,Monday,Sam Mendes,0
3,43600,250.00,112,1084.94,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,0
4,43602,258.00,115,890.87,Spider-Man 3,5.9,3576,2007,May,Tuesday,Sam Raimi,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1460,48363,0.00,3,0.32,The Last Waltz,7.9,64,1978,May,Monday,Martin Scorsese,0
1461,48370,0.03,19,3.15,Clerks,7.4,755,1994,Sep,Tuesday,Kevin Smith,0
1462,48375,0.00,7,0.00,Rampage,6.0,131,2009,Aug,Friday,Uwe Boll,0
1463,48376,0.00,3,0.00,Slacker,6.4,77,1990,Jul,Friday,Richard Linklater,0


Notice how this is similar to using vectorization in Numpy

We thus can use `apply` to use a function throughout a column

Can we **use apply on multiple columns**?

Say,
#### How to find sum of revenue and budget per movie?

In [ ]:
data[['revenue', 'budget']].apply(np.sum)

revenue    209867.04
budget      70353.62
dtype: float64

We can pass **multiple cols by packing them** within `[]`

But there's a mistake here. We wanted our results per movie (per row)

But, we are getting the sum of the columns

#### How can we use apply to work on individual rows?

In [ ]:
data[['revenue', 'budget']].apply(np.sum, axis=1)

0       3024.97
1       1261.00
2       1125.67
3       1334.94
4       1148.87
         ...   
1460       0.32
1461       3.18
1462       0.00
1463       0.00
1464       2.26
Length: 1465, dtype: float64

Every row of `revenue` was added to same row of `budget`

#### What does this `axis` mean in apply ?
  - If **axis = 0**, it will apply to **each column**, if **axis = 1**, **each row**
  - By default axis = 0

=> `apply()` can be applied on any dataframe along any particular axis


#### Similarly, how can I find profit per movie (revenue-budget)?

In [ ]:
def prof(x): # We define a function to calculate profit
  return x['revenue']-x['budget']
data['profit'] = data[['revenue', 'budget']].apply(prof, axis = 1)
data

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender,profit
0,43597,237.00,150,2787.97,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,0,2550.97
1,43598,300.00,139,961.00,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,0,661.00
2,43599,245.00,107,880.67,Spectre,6.3,4466,2015,Oct,Monday,Sam Mendes,0,635.67
3,43600,250.00,112,1084.94,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,0,834.94
4,43602,258.00,115,890.87,Spider-Man 3,5.9,3576,2007,May,Tuesday,Sam Raimi,0,632.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,48363,0.00,3,0.32,The Last Waltz,7.9,64,1978,May,Monday,Martin Scorsese,0,0.32
1461,48370,0.03,19,3.15,Clerks,7.4,755,1994,Sep,Tuesday,Kevin Smith,0,3.12
1462,48375,0.00,7,0.00,Rampage,6.0,131,2009,Aug,Friday,Uwe Boll,0,0.00
1463,48376,0.00,3,0.00,Slacker,6.4,77,1990,Jul,Friday,Richard Linklater,0,0.00


Thus, we can access the columns by their names inside the functions too using apply

### This was all about Pandas for today

#### Pandas is also a very vast library

- You can explore other methods for performing different tasks on your own


- We'll cover a few more important concepts in the next lecture


- We'll also do some practice questions using Pandas

***
